# 프로젝트: Movielens 영화 추천 

### 1) 데이터 준비와 전처리

Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 like로 바꿉니다.
ratings.rename(columns={'rating':'like'}, inplace=True)

In [4]:
ratings['like']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: like, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
#movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = "ISO-8859-1")
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 검색하기 쉽게 하기위해 타이틀 문자열을 소문자로 변경
movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


#### 여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.



### 2) 분석해 봅시다.

### ratings에 있는 유니크한 영화 개수

In [7]:
ratings['movie_id'].nunique()

3628

### rating에 있는 유니크한 사용자 수

In [8]:
ratings['user_id'].nunique()

6039

In [9]:
#movie_id 를 영화title 변경 하여 가장 인기 있는 영화에 타이틀이 나오게 했다.
ratings['movie_id'] = ratings['movie_id'].replace(movies['title'])

### 가장 인기 있는 영화 30개(인기순)

In [10]:
# 인기 많은 영화 30개  평점 순으로 해보았다.
movie_count_rating = ratings.groupby('movie_id')['like'].mean()
movie_count_rating.sort_values(ascending=False).head(30)

movie_id
passion in the desert (1998)                                                         5.000000
perils of pauline, the (1947)                                                        5.000000
cup, the (phörpa) (1999)                                                             5.000000
old lady who walked in the sea, the (vieille qui marchait dans la mer, la) (1991)    5.000000
mchale's navy (1997)                                                                 5.000000
eraserhead (1977)                                                                    5.000000
guess who's coming to dinner (1967)                                                  5.000000
grease 2 (1982)                                                                      5.000000
kurt & courtney (1998)                                                               5.000000
in the company of men (1997)                                                         5.000000
bluebeard (1944)                                   

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다

In [11]:
data = ratings[['user_id','movie_id','like']]

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toystroy (1995)' , 'popeye (1980)'  ,'aerican pop (1981)' ,'jumanji (1995)' ,'kansas city (1996)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['zoo']*5, 'movie_id': my_favorite, 'like':[5]*5})

if not data.isin({'user_id':['zoo']})['user_id'].any():  # user_id에 'zoo'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,like
1000203,6040,"leopard son, the (1996)",3
1000205,6040,bird of prey (1996),5
1000206,6040,naked in new york (1994),5
1000207,6040,palookaville (1996),4
1000208,6040,"associate, the (1996)",4
0,zoo,toystroy (1995),5
1,zoo,popeye (1980),5
2,zoo,aerican pop (1981),5
3,zoo,jumanji (1995),5
4,zoo,kansas city (1996),5


In [13]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['zoo'])     
print(movie_to_idx['popeye (1980)'])

6039
980


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,movie_id,like
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3628,5
1,6039,980,5
2,6039,3629,5
3,6039,40,5


### 4) CSR matrix를 직접 만들어 봅시다.

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.like, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3630 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, iterations=20, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3630x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [21]:
zoo, toystroy = user_to_idx['zoo'], movie_to_idx['popeye (1980)']
zoo_vector, popeye_vector = als_model.user_factors[zoo], als_model.item_factors[toystroy]


In [22]:
zoo_vector

array([-0.1624903 ,  0.2068286 ,  0.515655  , -0.30343875,  0.14021339,
       -0.28354266,  0.08410589,  0.12780634, -0.1306502 ,  0.04630427,
       -0.4342489 , -0.03890776,  0.3012268 , -0.01329238, -0.17067392,
       -0.05629938,  0.07339306, -0.6370355 , -0.09480707,  0.5261815 ,
       -0.16875806,  0.13598412,  0.2776705 , -0.24824409, -0.03178801,
       -0.25981733, -0.150441  ,  0.2298823 , -0.17018387, -0.0335166 ,
        0.19251317, -0.6048115 ,  0.15780841, -0.08985134,  0.07896076,
       -0.14293824, -0.13133883,  0.25602728, -0.5487447 , -0.30372277,
        0.38988346,  0.65920156,  0.46160015, -0.25222856,  0.11270072,
       -0.07093859, -0.04705682,  0.19798517, -0.10463718,  0.09657033,
        0.19424312, -0.12686141, -0.37005022,  0.3544454 ,  0.17213494,
        0.09639302,  0.19088727,  0.15863846, -0.20772046,  0.03542969,
        0.11869504, -0.09007472, -0.08703193, -0.22531757,  0.19546503,
        0.09334365,  0.06637633,  0.13096534,  0.05801593, -0.32

In [23]:
popeye_vector

array([ 1.26245916e-02,  9.44816321e-03,  2.60582808e-02,  2.84195296e-04,
        2.76325922e-03, -1.30607025e-03, -6.01564161e-03,  4.44279332e-03,
       -1.41951232e-03,  4.25372133e-03, -1.21268677e-02,  2.07383325e-03,
        1.81192122e-02,  1.99036658e-04, -8.86714645e-03, -4.81862947e-03,
        9.85904131e-04, -7.30320485e-03, -1.76338051e-02,  1.99086796e-02,
        2.23438945e-02,  8.06891825e-03,  1.24615300e-02,  2.11990792e-02,
        9.11583565e-03,  7.41563737e-03, -1.50703397e-02,  1.37928315e-02,
        7.71318749e-03, -8.71295109e-04,  1.17193218e-02, -1.22775361e-02,
        7.45368144e-03, -1.94814126e-03,  1.70266267e-03, -4.63552168e-03,
       -1.46834217e-02,  1.24850925e-02, -1.83635075e-02,  1.56001886e-03,
        1.93505809e-02,  1.61063280e-02,  1.55267362e-02,  1.09147867e-02,
        9.66860820e-03,  9.90726147e-03,  2.86998926e-03, -6.43747440e-03,
        3.43841547e-03,  1.07385376e-02, -2.21275142e-03, -6.92544179e-03,
       -5.29399933e-03,  

In [24]:
np.dot(zoo_vector, popeye_vector)

0.7612913

In [25]:
toystroy = movie_to_idx['popeye (1980)']
movie_vector = als_model.item_factors[toystroy]
np.dot(zoo_vector, popeye_vector)

0.7612913

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [26]:
favorite_movie = 'popeye (1980)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(980, 1.0000001),
 (2430, 0.4555578),
 (70, 0.44469607),
 (1460, 0.44446233),
 (2511, 0.4423599),
 (69, 0.4366845),
 (3629, 0.43331015),
 (3165, 0.43324494),
 (3628, 0.4316699),
 (3285, 0.4284082),
 (3556, 0.42822716),
 (3504, 0.42759442),
 (3499, 0.42624053),
 (2699, 0.42551404),
 (3619, 0.42529556)]

In [27]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['popeye (1980)',
 'bats (1999)',
 'trust (1990)',
 'magnolia (1999)',
 'never been kissed (1999)',
 'shampoo (1975)',
 'aerican pop (1981)',
 'life of émile zola, the (1937)',
 'toystroy (1995)',
 'wilde (1997)',
 'walking and talking (1996)',
 'g. i. blues (1960)',
 'someone to watch over me (1987)',
 'when the cats away (chacun cherche son chat) (1996)',
 'anchors aweigh (1945)']

In [28]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [29]:
get_similar_movie('popeye (1980)')

['popeye (1980)',
 'bats (1999)',
 'trust (1990)',
 'magnolia (1999)',
 'never been kissed (1999)',
 'shampoo (1975)',
 'aerican pop (1981)',
 'life of émile zola, the (1937)',
 'toystroy (1995)',
 'wilde (1997)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [30]:
user = user_to_idx['zoo']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(380, 0.2662406),
 (70, 0.17931929),
 (1292, 0.10423398),
 (6, 0.1023923),
 (50, 0.09874652),
 (1428, 0.094304256),
 (604, 0.08853073),
 (1460, 0.087914914),
 (88, 0.08530977),
 (149, 0.08390893),
 (69, 0.083607435),
 (1590, 0.081752256),
 (2200, 0.08084732),
 (197, 0.08010453),
 (487, 0.0775172),
 (273, 0.076525174),
 (778, 0.07612009),
 (8, 0.075766794),
 (793, 0.07552196),
 (817, 0.0754364)]

In [31]:
[idx_to_movie[i[0]] for i in movie_recommended]

['braindead (1992)',
 'trust (1990)',
 'better off dead... (1985)',
 'when harry met sally... (1989)',
 'third miracle, the (1999)',
 'fried green tomatoes (1991)',
 'gigi (1958)',
 'magnolia (1999)',
 'pope of greenwich village, the (1984)',
 'alley cats, the (1968)',
 'shampoo (1975)',
 'beyond the poseidon adventure (1979)',
 'unforgettable (1996)',
 'evening star, the (1996)',
 'day the earth stood still, the (1951)',
 'taming of the shrew, the (1967)',
 'lay of the land, the (1997)',
 'window to paris (1994)',
 'head on (1998)',
 'message to love: the isle of wight festival (1996)']

### 9) 선호도 및 유사도 기여도 확인하기

In [32]:
aericapop = movie_to_idx['aerican pop (1981)']
explain = als_model.explain(user, csr_data, itemid=aericapop)

In [33]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('jumanji (1995)', 0.00633760800456497),
 ('popeye (1980)', 0.006009951106942892),
 ('toystroy (1995)', 0.005997003583669949),
 ('aerican pop (1981)', 0.005453884839198935),
 ('kansas city (1996)', 0.005125776866741257)]

# 프로젝트를 마치며... 

### 데이터 전처리 부분 아직 많이 미숙한것같다 개념들은 알아가는데 항상 사용하려구 하면 헷갈리고 어렵다.... 활용하고 응용 하기까지 더 많이 검색해보고 사용해 봐야 할것같다. 
### 데이터 전처리 이후는 크게 어려운 부분이 없었다. 이번 노드는 잘 읽으면서 따라가면서 하기 충분하였다. 